# Data Cleaning
    Quinn Theobald
    INFO 2950
    Final Project

`First, let's load up the data set.`

In [1]:
import pandas
import numpy
import re

In [2]:
wikipeople = pandas.read_csv('wikipeople.csv')
wikipeople.head()

,name,birth_date,occupation,education
0,Alain Connes,1947,NaN,"['École Normale Supérieure', 'Pierre and Marie..."
1,Aldous Huxley,1894,| occupation = {{hlist|Writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
2,Arthur Schopenhauer,1788,NaN,[]
3,Albert Einstein,1879,NaN,"['ETH Zurich|Federal Polytechnic School', 'Zur..."
4,Alfred Korzybski,1879,NaN,['Warsaw University of Technology']


To do list:
* convert `birth_date` to type int
* try to wrangle with `occupations`
* eliminate `Batchelor of Arts`
* make `name` the index

## Initial Cleaning

#### Convert birth_date to type int

In [3]:
wikipeople['birth_date'] = wikipeople['birth_date'].astype('int64')
wikipeople.head()

ValueError: invalid literal for int() with base 10: '| birth_date  = Late 12th Century'

Ruh roh. There seems to still be non-numbers in the `birth_date` column.

In [4]:
def only_years(thing):
    if thing is numpy.nan:
        return ''
    if len(thing) > 4:
        return ''
    else:
        return thing

In [5]:
wikipeople['birth_date'] = [ only_years(date) for date in wikipeople['birth_date'] ]
wikipeople.head()

,name,birth_date,occupation,education
0,Alain Connes,1947,NaN,"['École Normale Supérieure', 'Pierre and Marie..."
1,Aldous Huxley,1894,| occupation = {{hlist|Writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
2,Arthur Schopenhauer,1788,NaN,[]
3,Albert Einstein,1879,NaN,"['ETH Zurich|Federal Polytechnic School', 'Zur..."
4,Alfred Korzybski,1879,NaN,['Warsaw University of Technology']


In [6]:
wikipeople['birth_date'] = wikipeople['birth_date'].astype('int64')
wikipeople.head()

ValueError: invalid literal for int() with base 10: ''

In [7]:
def empty_to_zero(thing):
    if thing == '':
        return 0
    else:
        return thing

In [8]:
wikipeople['birth_date'] = [ empty_to_zero(date) for date in wikipeople['birth_date'] ]
wikipeople['birth_date'] = wikipeople['birth_date'].astype('int64')
wikipeople.dtypes['birth_date']

dtype('int64')

#### Education field
    Now let's take a closer look at the education field.

In [9]:
interesting = []
count = 0
for school in wikipeople.education:
    count += 1
    if (count > 30000):
        break
    if school.find('|') > -1:
        interesting.append(school)
    elif school.find('achelor') > -1:
        # I decided to exclude the 'b' in 'bachelor' for in case the b is sometimes capitalized, sometimes not
        interesting.append(school)
    elif school.find('asters') > -1:
        interesting.append(school)
interesting

["['ETH Zurich|Federal Polytechnic School', 'Zurich', 'University of Zurich']",
 "['École Polytechnique|École Polytech', 'École des ponts ParisTech|École des Ponts']",
 "['Malvern College', 'Tonbridge School', 'Eastbourne College', 'Trinity College, Cambridge', 'occultist', 'ceremonial magic', 'Thelema', 'Æon of Horus', 'Royal Leamington Spa', 'Warwickshire', 'Christian fundamentalism|fundamentalist Christian', 'Plymouth Brethren', 'Western esotericism', 'Trinity College, Cambridge|Trinity College', 'University of Cambridge', 'British intelligence agency', 'Hermetic Order of the Golden Dawn', 'Samuel Liddell MacGregor Mathers', 'Charles Henry Allan Bennett|Allan Bennett', 'Boleskine House', 'Loch Ness', 'Oscar Eckenstein', 'Hindu', 'Buddhist', 'Rose Edith Kelly', 'Cairo', 'Aiwass', 'The Book of the Law', 'True Will', 'Magick (Thelema)|magick', '1905 Kanchenjunga expedition|an unsuccessful attempt to climb Kanchenjunga', 'George Cecil Jones', 'A∴A∴', 'Ordo Templi Orientis', 'First World

So... we've got some wierd stuff in there. We're going to keep that in mind for later. It looks like Bachelor of Arts is going to be harder to eliminate than I thought, so we will leave it in for now. In the end, this will likely be fine as I plan on doing a value count of the top schools. If *Bachelor of Arts* shows up as a top school, we can just filter it out. Don't worry, I won't be fooled into thinking *Bachelor of Arts* is an institution.

#### Wrangling with `occupation`

In [10]:
wikipeople_employed = wikipeople[pandas.notnull(wikipeople['occupation'])]
wikipeople_employed.head()

,name,birth_date,occupation,education
1,Aldous Huxley,1894,| occupation = {{hlist|Writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
5,Alfred Hitchcock,1899,| occupation = {{hlist|Film director|film prod...,"['Salesian College, Battersea', ""St Ignatius' ..."
6,Alfred Nobel,1833,"| occupation = Chemist, engineer, inve...",[]
7,Alexander Graham Bell,1847,| occupation = {{hlist |Inventor |Scientist}} ...,"['University of Edinburgh', 'University Colleg..."
10,A. A. Milne,1882,"| occupation = Novelist, playwright, poet","['Trinity College, Cambridge']"


Let's get rid of some of this extra text. We are making sure to do this in the `wikipeople` dataframe, and then we will recreate the `wikipeople_occupied` dataframe again later to take a look.

In [11]:
def erase1(thing):
    if 'occupation' in thing:
        if pandas.isnull(re.search('.*=(.*)',thing)):
            return ''
        return re.search('.*=(.*)',thing).group(1)
    else:
        return thing

In [12]:
wikipeople_employed.dtypes

name          object
birth_date     int64
occupation    object
education     object
dtype: object

first we have to convert the column to type `str`

In [13]:
wikipeople['occupation'] = wikipeople['occupation'].astype('str')

In [14]:
wikipeople['occupation'] = [ erase1(job) for job in wikipeople['occupation'] ]
wikipeople.head()

,name,birth_date,occupation,education
0,Alain Connes,1947,nan,"['École Normale Supérieure', 'Pierre and Marie..."
1,Aldous Huxley,1894,{{hlist|Writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
2,Arthur Schopenhauer,1788,nan,[]
3,Albert Einstein,1879,nan,"['ETH Zurich|Federal Polytechnic School', 'Zur..."
4,Alfred Korzybski,1879,nan,['Warsaw University of Technology']


And then let's make everything lowercase.

In [15]:
wikipeople['occupation'] = [job.lower() for job in wikipeople['occupation']]
wikipeople.head()

,name,birth_date,occupation,education
0,Alain Connes,1947,nan,"['École Normale Supérieure', 'Pierre and Marie..."
1,Aldous Huxley,1894,{{hlist|writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
2,Arthur Schopenhauer,1788,nan,[]
3,Albert Einstein,1879,nan,"['ETH Zurich|Federal Polytechnic School', 'Zur..."
4,Alfred Korzybski,1879,nan,['Warsaw University of Technology']


Notice how *'Writer'* is now *'writer'*? <br>
Let's look at the rows that have occupations.

In [16]:
wikipeople_employed_temp = wikipeople[wikipeople['occupation'] != 'nan']
wikipeople_employed = wikipeople_employed_temp[wikipeople_employed_temp['occupation'] != '']
wikipeople_employed.head()

,name,birth_date,occupation,education
1,Aldous Huxley,1894,{{hlist|writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
5,Alfred Hitchcock,1899,{{hlist|film director|film producer|actor|scr...,"['Salesian College, Battersea', ""St Ignatius' ..."
6,Alfred Nobel,1833,"chemist, engineer, inventor, businessman, ph...",[]
7,Alexander Graham Bell,1847,&quot;n&quot;}}}},"['University of Edinburgh', 'University Colleg..."
10,A. A. Milne,1882,"novelist, playwright, poet","['Trinity College, Cambridge']"


Looking much better. The `occupation` column is still ugly, but now we can pick our top occupations and search the `occupation` strings for those occupations to make dummy variables later on.

#### Setting `name` field as index

In [17]:
wikipeople = wikipeople.set_index('name')

In [18]:
wikipeople.head()

,birth_date,occupation,education
name,,,
Alain Connes,1947,nan,"['École Normale Supérieure', 'Pierre and Marie..."
Aldous Huxley,1894,{{hlist|writer|philosopher}},"['Eton College', 'Balliol College, Oxford']"
Arthur Schopenhauer,1788,nan,[]
Albert Einstein,1879,nan,"['ETH Zurich|Federal Polytechnic School', 'Zur..."
Alfred Korzybski,1879,nan,['Warsaw University of Technology']


Huh. I was scared of duplicate names, but it seems wikipedia has already banned pages with the same title. This makes sense for the site, and sure is nice for my dataset.

## College Count Dataframe
    Now, we will construct another dataframe representing each college and their count.

#### Arrays to pass into the Dataframe

In [19]:
wikipeople.dtypes

birth_date     int64
occupation    object
education     object
dtype: object

In [20]:
wikipeople['education'][0]

"['École Normale Supérieure', 'Pierre and Marie Curie University']"

Ruh roh. I thought I had made the education field all arrays, but it looks like they are strings... this is bad.

In [21]:
def string_to_array(thing):
    # if it is already an array, return
    if isinstance(thing, list):
        return thing
    # else turn it into an array
    array = []
    while len(thing) > 2:
        temp = ' '
        r = re.search('\'([^,]*)\'', thing)
        if r:
            temp = r.group(1)
            array.append(temp)
        else:
            return array
        thing = thing[len(temp)+4:]
    return array

In [22]:
string_to_array( wikipeople['education'][0] )

['École Normale Supérieure', 'Pierre and Marie Curie University']

In [23]:
wikipeople_temp = wikipeople

In [24]:
wikipeople_temp['education'] = [ string_to_array(schools) for schools in wikipeople['education'] ]
wikipeople_temp

,birth_date,occupation,education
name,,,
Alain Connes,1947,nan,"[École Normale Supérieure, Pierre and Marie Cu..."
Aldous Huxley,1894,{{hlist|writer|philosopher}},[Eton College]
Arthur Schopenhauer,1788,nan,[]
Albert Einstein,1879,nan,"[ETH Zurich|Federal Polytechnic School, Zurich..."
Alfred Korzybski,1879,nan,[Warsaw University of Technology]
...,...,...,...
Liezl van der Merwe,1980,member of parliament,[]
Andrea Romero,0,nan,"[Stanford University, Bachelor of Arts|BA]"
Joy Garratt,1952,nan,"[Excelsior College, Bachelor of Arts|BA, Unive..."


In [25]:
# the list of colleges attended by some famous person
colleges = []
for schools in wikipeople['education']:
    for school in schools:
        if school not in colleges:
            colleges.append(school)
colleges

['École Normale Supérieure',
 'Pierre and Marie Curie University',
 'Eton College',
 'ETH Zurich|Federal Polytechnic School',
 'Zurich',
 'University of Zurich',
 'Warsaw University of Technology',
 'University of Edinburgh',
 'University College London',
 'St. Petersburg University',
 'New York University',
 'Technical University of Berlin',
 'Technical University of Munich',
 'Karlsruhe Institute of Technology',
 'University of Algiers',
 'Columbia University',
 'University of Strasbourg',
 'University of Lyon',
 'Lycée Henri-IV',
 'École Polytechnique|École Polytech',
 'École des ponts ParisTech|École des Ponts',
 'Malvern College',
 'Tonbridge School',
 'Eastbourne College',
 'occultist',
 'ceremonial magic',
 'Thelema',
 'Æon of Horus',
 'Royal Leamington Spa',
 'Warwickshire',
 'Christian fundamentalism|fundamentalist Christian',
 'Plymouth Brethren',
 'Western esotericism',
 'University of Cambridge',
 'British intelligence agency',
 'Hermetic Order of the Golden Dawn',
 'Samuel

In [26]:
# the number of people that attended each college
college_count = []
# an array with the names of each person that attended the college (so we can reference the other DataFrame)
college_students = []
for college in colleges:
    college_count.append(0)
    college_students.append([])

#### Construct the Dataframe

In [27]:
tertiary_education = pandas.DataFrame({'school':colleges,'count':college_count,'notable_students':college_students})
tertiary_education = tertiary_education.set_index('school')
tertiary_education.head()

,count,notable_students
school,,
École Normale Supérieure,0,[]
Pierre and Marie Curie University,0,[]
Eton College,0,[]
ETH Zurich|Federal Polytechnic School,0,[]
Zurich,0,[]


Let's fill her up with data.

In [28]:
# testing index function
print (college_count[colleges.index('Cornell University')])
colleges.index('Cornell University')

0


133

In [32]:
for index in wikipeople.index:
    for school in wikipeople['education'][index]:
        college_count[colleges.index(school)] += 1
        college_students[colleges.index(school)].append(index)

In [33]:
# testing count
print (college_count[colleges.index('Cornell University')])
colleges.index('Cornell University')

1759


133

We've had 1759 wikipedia-level famous people attend Cornell! I wonder if this is a lot or a little in comparison to other schools.

In [34]:
tertiary_education = pandas.DataFrame({'school':colleges,'count':college_count,'notable_students':college_students})
tertiary_education = tertiary_education.set_index('school')
tertiary_education.head()

,count,notable_students
school,,
École Normale Supérieure,199,"[Alain Connes, André Weil, Henri Bergson, Jean..."
Pierre and Marie Curie University,48,"[Alain Connes, Ahmad Motamedi, Pierre-Louis Li..."
Eton College,702,"[Aldous Huxley, A. J. Ayer, George Orwell, Hor..."
ETH Zurich|Federal Polytechnic School,1,[Albert Einstein]
Zurich,15,"[Albert Einstein, Claus Westermann, Annemarie ..."


The really cool part is that now to access the rest of the data, we just need to use an element of the `notable_students` array as the index of the wikipeople dataframe to get their birth_date, occupation, or other schools.

#### Save dataframes

Let's save the work we've done so we can load it up in other notebooks.

In [35]:
wikipeople.head()

,birth_date,occupation,education
name,,,
Alain Connes,1947,nan,"[École Normale Supérieure, Pierre and Marie Cu..."
Aldous Huxley,1894,{{hlist|writer|philosopher}},[Eton College]
Arthur Schopenhauer,1788,nan,[]
Albert Einstein,1879,nan,"[ETH Zurich|Federal Polytechnic School, Zurich..."
Alfred Korzybski,1879,nan,[Warsaw University of Technology]


In [36]:
tertiary_education.head()

,count,notable_students
school,,
École Normale Supérieure,199,"[Alain Connes, André Weil, Henri Bergson, Jean..."
Pierre and Marie Curie University,48,"[Alain Connes, Ahmad Motamedi, Pierre-Louis Li..."
Eton College,702,"[Aldous Huxley, A. J. Ayer, George Orwell, Hor..."
ETH Zurich|Federal Polytechnic School,1,[Albert Einstein]
Zurich,15,"[Albert Einstein, Claus Westermann, Annemarie ..."


In [38]:
wikipeople.to_csv('wikipeople_cleaned.csv',index=True)

In [39]:
tertiary_education.to_csv('tertiary_education.csv',index=True)

That's all, folks! We now have data in two very neat DataFrames. Next up: we do some analysis with this well-formatted data tables!